In [1]:
pip install --upgrade keras

Requirement already up-to-date: keras in /usr/local/lib/python3.6/dist-packages (2.2.4)


In [0]:
import random as rd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

def normalization(data, momentum=0.99, epsilon=1e-4):
  mean = np.mean(data)
  variance = np.var(data)
  
  return momentum * ((data - mean) / np.sqrt(variance**2 + 1e-5)) + epsilon

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/datas

In [0]:
LEARN_RATE = 1e-4
BATCH_SIZE = 64
TOTAL_EPOCH = 100
NUM_HIDDEN = 256
NUM_INPUT  = 28 * 28

class Autoencoder:
    def __init__(self, sess, input_size, hidden_size, learn_rate=1e-4, name="main"):
        self.sess = sess
        self._name = name
        self._input_size = input_size
        self._hidden_size = hidden_size
        self._learn_rate = learn_rate

        self._buildNet()

    def _buildNet(self):
      with tf.device('/device:GPU:0'):
        with tf.variable_scope(self._name):
          self.x = tf.placeholder(tf.float32, [None, self._input_size], name='input_x')
#         print("x:", self.x.shape)

#           with tf.name_scope("Encoder"):
#             self.encoder = tf.keras.layers.Dense(self._hidden_size, activation = tf.nn.relu,
#                                                 kernel_initializer=tf.keras.initializers.he_normal())(self.x)
# #              encoder = tf.keras.layers.Dense(2 * self._hidden_size, activation = tf.nn.relu)(encoder)
# #              print("encoder:", encoder.shape)

#           with tf.name_scope("Decoder"):
# #              self.decoder = tf.keras.layers.Dense(self._hidden_size, activation = tf.nn.relu)(encoder)
#             self.decoder = tf.keras.layers.Dense(self._input_size, activation = tf.nn.relu,
#                                                 kernel_initializer=tf.keras.initializers.he_normal())(self.encoder)
# #              print("decoder", self.decoder.shape)
          with tf.name_scope("Encoder"):
            W_encode = tf.get_variable('W_encode', shape=[NUM_INPUT, NUM_HIDDEN],
                                        initializer = tf.keras.initializers.he_normal())
            b_encode = tf.Variable(tf.random_normal([NUM_HIDDEN]))

            self.encoder = tf.nn.relu(tf.add(tf.matmul(self.x, W_encode), b_encode))

          with tf.name_scope("Decoder"):
            W_decode = tf.get_variable('W_decode', shape=[NUM_HIDDEN, NUM_INPUT], 
                                        initializer = tf.keras.initializers.he_normal())
            b_decode = tf.Variable(tf.random_normal([NUM_INPUT]))
            self.decoder = tf.nn.relu(tf.add(tf.matmul(self.encoder, W_decode), b_decode))

          with tf.name_scope("Cost"):
            print("x.shape:", self.x.shape)
            print("decoder.shape:", self.decoder.shape)
            self.cost = tf.reduce_mean(tf.pow(tf.subtract(self.x, self.decoder), 2))

          with tf.name_scope("Train"):
            self.optimizer = tf.train.RMSPropOptimizer(self._learn_rate, epsilon=1e-4, momentum=0.99).minimize(self.cost)

    def predict(self, minibatch):
      feed = {
          self.x: minibatch
      }
        
      return self.sess.run(self.decoder, feed)

    def train(self, trainbatch):       
      feed = {
          self.x: trainbatch
      }

      return self.sess.run([self.cost, self.optimizer], feed)


In [5]:
TOTAL_BATCH = int(mnist.train.num_examples / BATCH_SIZE)

init = tf.global_variables_initializer()
sess = tf.Session()
model = Autoencoder(sess, NUM_INPUT, NUM_HIDDEN)

sess.run(init)

Instructions for updating:
Colocations handled automatically by placer.
x.shape: (?, 784)
decoder.shape: (?, 784)


In [6]:
for epoch in range(TOTAL_EPOCH):
  total_cost = 0
  
  for i in range(TOTAL_BATCH):
    batch_xs, batch_ys = mnist.train.next_batch(BATCH_SIZE)
    batch_xs = normalization(batch_xs)
    
    cost_val, _ = model.train(batch_xs)
    total_cost += cost_val
  print("Epoch: {}    Avg_Cost: {}".format(epoch+1, total_cost/TOTAL_BATCH))

print("Optimizing Done!")

FailedPreconditionError: ignored

In [0]:
SAMPLE_SIZE = 10
RANDOM_N = rd.randint(0, 9989)

x_test = mnist.test.images[RANDOM_N: RANDOM_N + SAMPLE_SIZE]
# x_test = normalization(x_test)

sample = model.predict(x_test)
labels = mnist.test.images[RANDOM_N: RANDOM_N + SAMPLE_SIZE]
fig, ax = plt.subplots(2, SAMPLE_SIZE, figsize=(SAMPLE_SIZE, 2))

for i in range(SAMPLE_SIZE):
  ax[0][i].set_axis_off()
  ax[1][i].set_axis_off()
  
  ax[0][i].imshow(np.reshape(labels[i], (28, 28)))
  ax[1][i].imshow(np.reshape(sample[i], (28, 28)))
  
plt.show()